In [ ]:
import tensorflow as tf

In [ ]:
import tensorflow_datasets as tfds

In [ ]:
(train_data,test_data),ds_info = tfds.load(name='food101',
                                           split=['train','validation'],
                                           shuffle_files=True,
                                           as_supervised=True,
                                           with_info=True)

In [ ]:
ds_info.features

FeaturesDict({
    'image': Image(shape=(None, None, 3), dtype=tf.uint8),
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=101),
})

In [ ]:
one_sample = train_data.take(1)

In [ ]:
for image,label in one_sample:
  print(label)

tf.Tensor(65, shape=(), dtype=int64)


In [ ]:
def preprocess_image(image,label,image_shape=224):
  image = tf.image.resize(image,[image_shape,image_shape])
  return tf.cast(image,tf.float32),label

In [ ]:
train_data = train_data.map(map_func=preprocess_image,num_parallel_calls=tf.data.AUTOTUNE)
train_data = train_data.shuffle(buffer_size=1000).batch(batch_size=32).prefetch(buffer_size=tf.data.AUTOTUNE)

test_data = test_data.map(map_func=preprocess_image,num_parallel_calls=tf.data.AUTOTUNE)
test_data = test_data.batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
def create_tensorboard_callback(dir_name,experiment_name,experiment_num):
  log_dir = dir_name+'/'+experiment_name+'/'+experiment_num
  callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
  return callback

In [ ]:
path = 'model_checkpoints/cp.ckpt'
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(path,
                                                      save_best_only=True,
                                                      monitor='val_acc',
                                                      save_weights_only=True)

In [ ]:
callback = create_tensorboard_callback('food101models','efficientnet','0')

In [ ]:
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy(policy='mixed_float16')

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: Tesla T4, compute capability 7.5


INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: Tesla T4, compute capability 7.5


In [ ]:
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable=False
i = tf.keras.layers.Input(shape=(224,224,3))
x = base_model(i,training=False)
x = tf.keras.layers.GlobalMaxPool2D()(x)
x = tf.keras.layers.Dense(101)(x)
x = tf.keras.layers.Activation('softmax',dtype=tf.float32)(x)
model = tf.keras.Model(inputs=(i),outputs=(x))

16719872/16705208 [==============================] - 0s 0us/step


In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_data,epochs=3,steps_per_epoch=len(train_data),
                    validation_data=(test_data),validation_steps=int(0.15*len(test_data)),
                    callbacks=[callback,
                               model_checkpoint])

Epoch 1/3


KeyboardInterrupt: ignored

In [ ]:
results = model.evaluate(test_data)
results

790/790 [==============================] - 52s 65ms/step - loss: 2.3323 - accuracy: 0.5937


[2.33229660987854, 0.5936633944511414]

In [ ]:
cloned_model = tf.keras.models.clone_model(model)
cloned_model.summary

<bound method Model.summary of <keras.engine.functional.Functional object at 0x7f31e44f0490>>

In [ ]:
path

'model_checkpoints/cp.ckpt'

In [ ]:
for layer in model.layers:
  layer.trainable =True

In [ ]:
callback = create_tensorboard_callback('food101models','efficientnet','1')

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", 
                                                  patience=3)

In [ ]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",  
                                                 factor=0.2, 
                                                 patience=2,
                                                 verbose=1, 
                                                 min_lr=1e-7)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history_101_food_classes_all_data_fine_tune = model.fit(train_data,
                                                        epochs=100,
                                                        steps_per_epoch=len(train_data),
                                                        validation_data=test_data,
                                                        validation_steps=int(0.15 * len(test_data)), 
                                                        callbacks=[callback, 
                                                                   early_stopping,
                                                                   reduce_lr])

Epoch 1/100
2368/2368 [==============================] - 362s 148ms/step - loss: 0.9765 - accuracy: 0.7402 - val_loss: 0.9484 - val_accuracy: 0.7338 - lr: 1.0000e-04
Epoch 2/100
2368/2368 [==============================] - 349s 147ms/step - loss: 0.5784 - accuracy: 0.8383 - val_loss: 0.9587 - val_accuracy: 0.7383 - lr: 1.0000e-04
Epoch 3/100
2368/2368 [==============================] - ETA: 0s - loss: 0.3152 - accuracy: 0.9087
Epoch 3: ReduceLROnPlateau reducing learning rate to 1.9999999494757503e-05.
2368/2368 [==============================] - 346s 146ms/step - loss: 0.3152 - accuracy: 0.9087 - val_loss: 1.0560 - val_accuracy: 0.7328 - lr: 1.0000e-04
Epoch 4/100
2368/2368 [==============================] - 346s 145ms/step - loss: 0.0667 - accuracy: 0.9836 - val_loss: 1.1517 - val_accuracy: 0.7630 - lr: 2.0000e-05
